# CSE-6242 - Team 157 - Group Project

__TODO:__  
1. Remove custom stop words like "CHORUS:" or "INTRO:"  
2. Remove other stop words.

In [1]:
# global assumption panel
EMBED_STRATEGY = 'GloVe' # ['Word2Vec', 'DistilBERT', 'GloVe']

In [2]:
# packages
from project_code import data_gathering
from embedding import word2vec, distilbert, glove
from models import han

c:\Users\nccru\anaconda3\envs\cse-6242-project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data Gathering

In [3]:
# read data and perform basic cleaning operations
lyrics = data_gathering.read_lyrics(n_rows = 100)

Lyrics Dataset: Shape = (100, 13)
	Columns = ['title', 'tag', 'artist', 'year', 'views', 'features', 'lyrics', 'id', 'language_cld3', 'language_ft', 'language', 'cleaned_lyrics', 'tokenized_text']


## Embedding

In [4]:
# generate embeddings using word2vec
# if EMBED_STRATEGY == 'Word2Vec':
#     embedding_model = word2vec.apply_word2vec(lyrics['tokenized_text'].tolist())
if EMBED_STRATEGY == 'GloVe':
    lyrics_embed = glove.create_glove_matrix(data = lyrics, target_col = 'cleaned_lyrics')
elif EMBED_STRATEGY == 'DistilBERT':
    lyrics_embed = distilbert.distilbert_embed_all_docs(data = lyrics, target_col = 'lyrics')

GloVe Embedded Lyrics: Shape = (7476, 300)


## Modeling

In [5]:
han_model = han.HANClassifier(
    input_dim = 768,
    hidden_dim = 256,
    output_dim = 10
)
han_model

HANClassifier(
  (rnn): GRU(768, 256, batch_first=True, bidirectional=True)
  (attention): Attention(
    (attention_weights): Linear(in_features=512, out_features=1, bias=False)
  )
  (fc): Linear(in_features=512, out_features=10, bias=True)
  (softmax): Softmax(dim=1)
)